In this notebook we will build a speech recognition model.  

Below we'll import the libraries we'll be using.

In [ ]:
import os
import librosa   #for audio processing
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

Next, we'll download the dataset of speech commands from tensorflow.

In [ ]:
!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz

Here, we unzip the file we downloaded from tensorflow.

In [ ]:
!mkdir speech_commands
!tar -C ./speech_commands -xf speech_commands_v0.01.tar.gz 

Let's plot the waveform of an example spoken command, `samples`.

In [ ]:
train_path = 'speech_commands/'
filename = train_path+'no/afe0b87d_nohash_0.wav'
samples, sample_rate = librosa.load(filename, sr = 16000)
fig = plt.figure(figsize=(14, 8))
ax1 = fig.add_subplot(211)
ax1.set_title('Raw signal of ' + filename)
ax1.set_xlabel('time')
ax1.set_ylabel('Amplitude')
ax1.plot(np.linspace(0, sample_rate/len(samples), sample_rate), samples)

Below we will play the the `samples` audio command.

In [ ]:
ipd.Audio(samples,rate=sample_rate,autoplay=True)

Below we load the data into `all_wavs` and their respective labels into `all_labs`.  The labels are either `yes` or `no`.

We'll also print the number of examples in `all_wavs`.

In [ ]:
import os

directory = 'speech_commands/'

all_wavs = []
all_labs = []
for label in ['yes', 'no']:
    print(label)
    wavs = [f for f in os.listdir(directory + label) if f.endswith('.wav')]
    for wav in wavs:
        samples, sample_rate = librosa.load(directory + label + '/' + wav, sr = 16000)
        if(len(samples)== 16000): 
            all_wavs.append(samples)
            all_labs.append(label)
print(len(all_wavs))

Below we split our training and test data.  `X_train` is our processed audio files for training and `y_train` are their labels.  `X_test` and `y_test` are our test audio files and their labels, respectively.

In [ ]:
from sklearn.model_selection import train_test_split
 
all_wavs = np.array(all_wavs).reshape(-1,16000,1)
all_labs = np.array([lab == 'yes' for lab in all_labs])
X_train, X_test, y_train, y_test = train_test_split(all_wavs,all_labs,test_size = 0.2)

In the following lines, we will build together the layers of our model for speech recognition.

In [ ]:
!pip install keras=='2.3.1'
from keras.layers import Conv1D, Input, MaxPooling1D, Flatten, Dense
from keras.models import Model
 
inputs = Input(shape=(16000,1))
 
#First Conv1D layer
conv = Conv1D(8,13, padding='valid', activation='relu', strides=1)(inputs)
conv = MaxPooling1D(3)(conv)
 
#Second Conv1D layer
conv = Conv1D(16, 11, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
 
#Third Conv1D layer
conv = Conv1D(32, 9, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
 
#Fourth Conv1D layer
conv = Conv1D(64, 7, padding='valid', activation='relu', strides=1)(conv)
conv = MaxPooling1D(3)(conv)
 
#Flatten layer
conv = Flatten()(conv)
 
#Dense Layer 1
conv = Dense(256, activation='relu')(conv)
 
#Dense Layer 2
conv = Dense(128, activation='relu')(conv)
 
outputs = Dense(1, activation='sigmoid')(conv)
 
model = Model(inputs, outputs)

We then `fit` the model.  We use a `mean_squared_error` `loss` and optimize the weigths using use `adam` as our `optimizer`. We iterate of the data 15 times.  Each time, or `epoch`, we print out the `accuracy` and `loss` of our model so far.

In [ ]:
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['accuracy'])
 
model.fit(X_train, y_train ,epochs=15, batch_size=32)

We then report the final `accuracy` and `loss` on the `X_test` and `y_test` data.

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_train.shape

In [ ]:
sum(sum(X_train))